<a href="https://colab.research.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Copyright 2018 The TensorFlow Hub Authors.

Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2018 The TensorFlow Hub Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

# BERT End to End (Fine-tuning + Predicting) in 5 minutes with Cloud TPU

## Overview

**BERT**, or **B**idirectional **E**mbedding **R**epresentations from **T**ransformers, is a new method of pre-training language representations which obtains state-of-the-art results on a wide array of Natural Language Processing (NLP) tasks. The academic paper can be found here: https://arxiv.org/abs/1810.04805.

This Colab demonstates using a free Colab Cloud TPU to fine-tune sentence and sentence-pair classification tasks built on top of pretrained BERT models and 
run predictions on tuned model. The colab demonsrates loading pretrained BERT models from both [TF Hub](https://www.tensorflow.org/hub) and checkpoints.

**Note:**  You will need a GCP (Google Compute Engine) account and a GCS (Google Cloud 
Storage) bucket for this Colab to run.

Please follow the [Google Cloud TPU quickstart](https://cloud.google.com/tpu/docs/quickstart) for how to create GCP account and GCS bucket. You have [$300 free credit](https://cloud.google.com/free/) to get started with any GCP product. You can learn more about Cloud TPU at https://cloud.google.com/tpu/docs.

This notebook is hosted on GitHub. To view it in its original repository, after opening the notebook, select **File > View on GitHub**.

## Learning objectives

In this notebook, you will learn how to train and evaluate a BERT model using TPU.

## Instructions

<h3><a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>  &nbsp;&nbsp;Train on TPU</h3>

   1. Create a Cloud Storage bucket for your TensorBoard logs at http://console.cloud.google.com/storage and fill in the BUCKET parameter in the "Parameters" section below.
 
   1. On the main menu, click Runtime and select **Change runtime type**. Set "TPU" as the hardware accelerator.
   1. Click Runtime again and select **Runtime > Run All** (Watch out: the "Colab-only auth for this notebook and the TPU" cell requires user input). You can also run the cells manually with Shift-ENTER.

### Set up your TPU environment

In this section, you perform the following tasks:

*   Set up a Colab TPU running environment
*   Verify that you are connected to a TPU device
*   Upload your credentials to TPU to access your GCS bucket.

In [2]:
import datetime
import json
import os
import pprint
import random
import string
import sys
import tensorflow as tf

assert 'COLAB_TPU_ADDR' in os.environ, 'ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!'
TPU_ADDRESS = 'grpc://' + os.environ['COLAB_TPU_ADDR']
print('TPU address is', TPU_ADDRESS)

from google.colab import auth
auth.authenticate_user()
with tf.Session(TPU_ADDRESS) as session:
  print('TPU devices:')
  pprint.pprint(session.list_devices())

  # Upload credentials to TPU.
  with open('/content/adc.json', 'r') as f:
    auth_info = json.load(f)
  tf.contrib.cloud.configure_gcs(session, credentials=auth_info)
  # Now credentials are set for all future sessions on this TPU.

TPU address is grpc://10.48.211.250:8470
TPU devices:
[_DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:CPU:0, CPU, -1, 1170319143181071364),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 17179869184, 7901937886271153117),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 15692900891580644366),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:1, TPU, 17179869184, 12579674385720456336),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:2, TPU, 17179869184, 14521091570843632124),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:3, TPU, 17179869184, 9050624561507508431),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:4, TPU, 17179869184, 1044936136023594616),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:5, TPU, 17179869184, 16194396579982804756),
 _DeviceAttributes(/job:tpu_worker/replica:0/task:0/device:TPU:6, TPU, 17179869184, 10540519373

### Prepare and import BERT modules
​
With your environment configured, you can now prepare and import the BERT modules. The following step clones the source code from GitHub and import the modules from the source. Alternatively, you can install BERT using pip (!pip install bert-tensorflow).

In [3]:
import sys

!test -d bert_repo || git clone https://github.com/google-research/bert bert_repo
if not 'bert_repo' in sys.path:
  sys.path += ['bert_repo']

# import python modules defined by BERT
import modeling
import optimization
import run_classifier
import run_classifier_with_tfhub
import tokenization

# import tfhub 
import tensorflow_hub as hub

W0408 03:22:07.526196 140215859427200 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


### Prepare for training

This next section of code performs the following tasks:

*  Specify task and download training data.
*  Specify BERT pretrained model
*  Specify GS bucket, create output directory for model checkpoints and eval results.




In [4]:
TASK = 'CoLA' #@param {type:"string"}
assert TASK in ('MRPC', 'CoLA'), 'Only (MRPC, CoLA) are demonstrated here.'

# Download glue data.
! test -d download_glue_repo || git clone https://gist.github.com/60c2bdb54d156a41194446737ce03e2e.git download_glue_repo
!python download_glue_repo/download_glue_data.py --data_dir='glue_data' --tasks=$TASK

TASK_DATA_DIR = 'glue_data/' + TASK
print('***** Task data directory: {} *****'.format(TASK_DATA_DIR))
!ls $TASK_DATA_DIR

BUCKET = 'dl_project_19s' #@param {type:"string"}
assert BUCKET, 'Must specify an existing GCS bucket name'
OUTPUT_DIR = 'gs://{}/bert-tfhub/models/{}'.format(BUCKET, TASK)
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))

# Available pretrained model checkpoints:
#   uncased_L-12_H-768_A-12: uncased BERT base model
#   uncased_L-24_H-1024_A-16: uncased BERT large model
#   cased_L-12_H-768_A-12: cased BERT large model
BERT_MODEL = 'uncased_L-12_H-768_A-12' #@param {type:"string"}
BERT_MODEL_HUB = 'https://tfhub.dev/google/bert_' + BERT_MODEL + '/1'

	Completed!
***** Task data directory: glue_data/CoLA *****
dev.tsv  original  test.tsv  train.tsv
***** Model output directory: gs://dl_project_19s/bert-tfhub/models/CoLA *****


Now let's load tokenizer module from TF Hub and play with it.

In [5]:
tokenizer = run_classifier_with_tfhub.create_tokenizer_from_hub_module(BERT_MODEL_HUB)
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

Instructions for updating:
Colocations handled automatically by placer.


W0408 03:22:12.912067 140215859427200 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0408 03:22:15.313919 140215859427200 saver.py:1483] Saver not created because there are no variables in the graph to restore


['this',
 'here',
 "'",
 's',
 'an',
 'example',
 'of',
 'using',
 'the',
 'bert',
 'token',
 '##izer']

Also we initilize our hyperprams, prepare the training data and initialize TPU config.

In [0]:
TRAIN_BATCH_SIZE = 32
EVAL_BATCH_SIZE = 8
PREDICT_BATCH_SIZE = 8
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
MAX_SEQ_LENGTH = 128
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 1000
SAVE_SUMMARY_STEPS = 500

processors = {
  "cola": run_classifier.ColaProcessor,
  "mnli": run_classifier.MnliProcessor,
  "mrpc": run_classifier.MrpcProcessor,
}
processor = processors[TASK.lower()]()
label_list = processor.get_labels()

# Compute number of train and warmup steps from batch size
train_examples = processor.get_train_examples(TASK_DATA_DIR)
num_train_steps = int(len(train_examples) / TRAIN_BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

# Setup TPU related config
tpu_cluster_resolver = tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)
NUM_TPU_CORES = 8
ITERATIONS_PER_LOOP = 1000

def get_run_config(output_dir):
  return tf.contrib.tpu.RunConfig(
    cluster=tpu_cluster_resolver,
    model_dir=output_dir,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    tpu_config=tf.contrib.tpu.TPUConfig(
        iterations_per_loop=ITERATIONS_PER_LOOP,
        num_shards=NUM_TPU_CORES,
        per_host_input_for_training=tf.contrib.tpu.InputPipelineConfig.PER_HOST_V2))


# Fine-tune and Run Predictions on a pretrained BERT Model from TF Hub

This section demonstrates fine-tuning from a pre-trained BERT TF Hub module and running predictions.


In [7]:
# Force TF Hub writes to the GS bucket we provide.
os.environ['TFHUB_CACHE_DIR'] = OUTPUT_DIR

model_fn = run_classifier_with_tfhub.model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  bert_hub_module_handle=BERT_MODEL_HUB
)

estimator_from_tfhub = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)


W0408 03:22:16.000349 140215859427200 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f86459de2f0>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://dl_project_19s/bert-tfhub/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.48.211.250:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8642032d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.48.211.250:8470', '_evaluation_master': 'grpc://10.48.211.250:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shards=8, n

I0408 03:22:16.006883 140215859427200 estimator.py:201] Using config: {'_model_dir': 'gs://dl_project_19s/bert-tfhub/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.48.211.250:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8642032d30>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.48.211.250:8470', '_evaluation_master': 'grpc://10.48.211.250:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(i

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0408 03:22:16.011363 140215859427200 tpu_context.py:202] _TPUContext: eval_on_tpu True


At this point, you can now fine-tune the model, evaluate it, and run predictions on it.

In [0]:
# Train the model
def model_train(estimator):
  print('MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...')
  # We'll set sequences to be at most 128 tokens long.
  train_features = run_classifier.convert_examples_to_features(
      train_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started training at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(train_examples)))
  print('  Batch size = {}'.format(TRAIN_BATCH_SIZE))
  tf.logging.info("  Num steps = %d", num_train_steps)
  train_input_fn = run_classifier.input_fn_builder(
      features=train_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=True,
      drop_remainder=True)
  estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
  print('***** Finished training at {} *****'.format(datetime.datetime.now()))



In [9]:
model_train(estimator_from_tfhub)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 8551


I0408 03:22:16.046535 140215859427200 run_classifier.py:774] Writing example 0 of 8551


INFO:tensorflow:*** Example ***


I0408 03:22:16.050781 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: train-0


...

INFO:tensorflow:Loss for final step: 0.003010477.


I0408 03:26:00.228933 140215859427200 estimator.py:359] Loss for final step: 0.003010477.


INFO:tensorflow:training_loop marked as finished


I0408 03:26:00.235343 140215859427200 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-04-08 03:26:00.240297 *****


In [0]:
def model_eval(estimator):
  # Eval the model.
  eval_examples = processor.get_dev_examples(TASK_DATA_DIR)
  eval_features = run_classifier.convert_examples_to_features(
      eval_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  print('***** Started evaluation at {} *****'.format(datetime.datetime.now()))
  print('  Num examples = {}'.format(len(eval_examples)))
  print('  Batch size = {}'.format(EVAL_BATCH_SIZE))

  # Eval will be slightly WRONG on the TPU because it will truncate
  # the last batch.
  eval_steps = int(len(eval_examples) / EVAL_BATCH_SIZE)
  eval_input_fn = run_classifier.input_fn_builder(
      features=eval_features,
      seq_length=MAX_SEQ_LENGTH,
      is_training=False,
      drop_remainder=True)
  result = estimator.evaluate(input_fn=eval_input_fn, steps=eval_steps)
  print('***** Finished evaluation at {} *****'.format(datetime.datetime.now()))
  output_eval_file = os.path.join(OUTPUT_DIR, "eval_results.txt")
  with tf.gfile.GFile(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
      print('  {} = {}'.format(key, str(result[key])))
      writer.write("%s = %s\n" % (key, str(result[key])))


In [11]:
model_eval(estimator_from_tfhub)

INFO:tensorflow:Writing example 0 of 1043


I0408 03:26:00.335679 140215859427200 run_classifier.py:774] Writing example 0 of 1043


INFO:tensorflow:*** Example ***


I0408 03:26:00.342945 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 801: gs://dl_project_19s/bert-tfhub/models/CoLA/model.ckpt-801


I0408 03:26:39.451937 140215859427200 estimator.py:2039] Saving 'checkpoint_path' summary for global step 801: gs://dl_project_19s/bert-tfhub/models/CoLA/model.ckpt-801


INFO:tensorflow:evaluation_loop marked as finished


I0408 03:26:39.856950 140215859427200 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-04-08 03:26:39.859002 *****
***** Eval results *****
  eval_accuracy = 0.8076923
  eval_loss = 1.0026538
  global_step = 801
  loss = 1.2258574


In [0]:
def model_predict(estimator):
  # Make predictions on a subset of eval examples
  prediction_examples = processor.get_dev_examples(TASK_DATA_DIR)[:PREDICT_BATCH_SIZE]
  input_features = run_classifier.convert_examples_to_features(prediction_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=True)
  predictions = estimator.predict(predict_input_fn)

  for example, prediction in zip(prediction_examples, predictions):
    print('text_a: %s\ntext_b: %s\nlabel:%s\nprediction:%s\n' % (example.text_a, example.text_b, str(example.label), prediction['probabilities']))


In [13]:
model_predict(estimator_from_tfhub) 

INFO:tensorflow:Writing example 0 of 8


I0408 03:26:40.215862 140215859427200 run_classifier.py:774] Writing example 0 of 8


INFO:tensorflow:*** Example ***


I0408 03:26:40.220618 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

text_a: The sailors rode the breeze clear of the rocks.
text_b: None
label:1
prediction:[0.00171233 0.9982876 ]

text_a: The weights made the rope stretch over the pulley.
text_b: None
label:1
prediction:[0.00192551 0.9980744 ]

text_a: The mechanical doll wriggled itself loose.
text_b: None
label:1
prediction:[0.00181264 0.99818736]

text_a: If you had eaten more, you would want less.
text_b: None
label:1
prediction:[0.00203112 0.99796885]

text_a: As you eat the most, you want the least.
text_b: None
label:0
prediction:[0.0191375  0.98086256]

text_a: The more you would want, the less you would eat.
text_b: None
label:0
prediction:[0.00834747 0.9916525 ]

text_a: I demand that the more John eat, the more he pays.
text_b: None
label:0
prediction:[0.546991 0.453009]

text_a: Mary listens to the Grateful Dead, she gets depressed.
text_b: None
label:1
prediction:[0.03014459 0.9698554 ]

INFO:tensorflow:prediction_loop marked as finished


I0408 03:27:08.759365 140215859427200 error_handling.py:93] prediction_loop marked as finished


INFO:tensorflow:Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


I0408 03:27:08.759867 140214538884864 error_handling.py:70] Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


W0408 03:27:08.765861 140215859427200 tf_logging.py:161] Reraising captured error
Exception ignored in: <generator object TPUEstimator.predict at 0x7f86496ce830>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 2500, in predict
    rendezvous.raise_errors()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 132, in raise_errors
    six.reraise(typ, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/six.py", line 693, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 101, in catch_errors
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 478, in _run_infeed
    session.run(self._enqueue_ops)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 929, 

# Fine-tune and run predictions on a pre-trained BERT model from checkpoints

Alternatively, you can also load pre-trained BERT models from saved checkpoints.

In [14]:
# Setup task specific model and TPU running config.
BERT_PRETRAINED_DIR = 'gs://cloud-tpu-checkpoints/bert/' + BERT_MODEL 
print('***** BERT pretrained directory: {} *****'.format(BERT_PRETRAINED_DIR))
!gsutil ls $BERT_PRETRAINED_DIR

CONFIG_FILE = os.path.join(BERT_PRETRAINED_DIR, 'bert_config.json')
INIT_CHECKPOINT = os.path.join(BERT_PRETRAINED_DIR, 'bert_model.ckpt')

model_fn = run_classifier.model_fn_builder(
  bert_config=modeling.BertConfig.from_json_file(CONFIG_FILE),
  num_labels=len(label_list),
  init_checkpoint=INIT_CHECKPOINT,
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps,
  use_tpu=True,
  use_one_hot_embeddings=True
)

OUTPUT_DIR = OUTPUT_DIR.replace('bert-tfhub', 'bert-checkpoints')
tf.gfile.MakeDirs(OUTPUT_DIR)

estimator_from_checkpoints = tf.contrib.tpu.TPUEstimator(
  use_tpu=True,
  model_fn=model_fn,
  config=get_run_config(OUTPUT_DIR),
  train_batch_size=TRAIN_BATCH_SIZE,
  eval_batch_size=EVAL_BATCH_SIZE,
  predict_batch_size=PREDICT_BATCH_SIZE,
)

***** BERT pretrained directory: gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12 *****
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_config.json
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.index
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/bert_model.ckpt.meta
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/checkpoint
gs://cloud-tpu-checkpoints/bert/uncased_L-12_H-768_A-12/vocab.txt


W0408 03:27:13.712399 140215859427200 estimator.py:1924] Estimator's model_fn (<function model_fn_builder.<locals>.model_fn at 0x7f8641e8d510>) includes params argument, but params are not passed to Estimator.


INFO:tensorflow:Using config: {'_model_dir': 'gs://dl_project_19s/bert-checkpoints/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.48.211.250:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8637822c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.48.211.250:8470', '_evaluation_master': 'grpc://10.48.211.250:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUConfig(iterations_per_loop=1000, num_shard

I0408 03:27:13.720293 140215859427200 estimator.py:201] Using config: {'_model_dir': 'gs://dl_project_19s/bert-checkpoints/models/CoLA', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.48.211.250:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f8637822c18>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.48.211.250:8470', '_evaluation_master': 'grpc://10.48.211.250:8470', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1, '_tpu_config': TPUCo

INFO:tensorflow:_TPUContext: eval_on_tpu True


I0408 03:27:13.724836 140215859427200 tpu_context.py:202] _TPUContext: eval_on_tpu True


Now, you can repeat the training, evaluation, and prediction steps.

In [15]:
model_train(estimator_from_checkpoints)

MRPC/CoLA on BERT base model normally takes about 2-3 minutes. Please wait...
INFO:tensorflow:Writing example 0 of 8551


I0408 03:27:13.746681 140215859427200 run_classifier.py:774] Writing example 0 of 8551


INFO:tensorflow:*** Example ***


I0408 03:27:13.755463 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: train-0


...

INFO:tensorflow:Loss for final step: 0.0044394536.


I0408 03:30:30.842174 140215859427200 estimator.py:359] Loss for final step: 0.0044394536.


INFO:tensorflow:training_loop marked as finished


I0408 03:30:30.845039 140215859427200 error_handling.py:93] training_loop marked as finished


***** Finished training at 2019-04-08 03:30:30.858346 *****


In [16]:
model_eval(estimator_from_checkpoints)

INFO:tensorflow:Writing example 0 of 1043


I0408 03:30:30.942763 140215859427200 run_classifier.py:774] Writing example 0 of 1043


INFO:tensorflow:*** Example ***


I0408 03:30:30.945526 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 801: gs://dl_project_19s/bert-checkpoints/models/CoLA/model.ckpt-801


I0408 03:31:04.723025 140215859427200 estimator.py:2039] Saving 'checkpoint_path' summary for global step 801: gs://dl_project_19s/bert-checkpoints/models/CoLA/model.ckpt-801


INFO:tensorflow:evaluation_loop marked as finished


I0408 03:31:05.131429 140215859427200 error_handling.py:93] evaluation_loop marked as finished


***** Finished evaluation at 2019-04-08 03:31:05.139430 *****
***** Eval results *****
  eval_accuracy = 0.8125
  eval_loss = 0.9553203
  global_step = 801
  loss = 1.0344187


In [17]:
model_predict(estimator_from_checkpoints)

INFO:tensorflow:Writing example 0 of 8


I0408 03:31:05.494724 140215859427200 run_classifier.py:774] Writing example 0 of 8


INFO:tensorflow:*** Example ***


I0408 03:31:05.499989 140215859427200 run_classifier.py:461] *** Example ***


INFO:tensorflow:guid: dev-0


...

text_a: The sailors rode the breeze clear of the rocks.
text_b: None
label:1
prediction:[0.00183955 0.9981604 ]

text_a: The weights made the rope stretch over the pulley.
text_b: None
label:1
prediction:[0.00206414 0.9979359 ]

text_a: The mechanical doll wriggled itself loose.
text_b: None
label:1
prediction:[0.00205964 0.99794036]

text_a: If you had eaten more, you would want less.
text_b: None
label:1
prediction:[0.00188731 0.9981127 ]

text_a: As you eat the most, you want the least.
text_b: None
label:0
prediction:[0.00545117 0.9945489 ]

text_a: The more you would want, the less you would eat.
text_b: None
label:0
prediction:[0.01841665 0.98158324]

text_a: I demand that the more John eat, the more he pays.
text_b: None
label:0
prediction:[0.23195992 0.7680401 ]

text_a: Mary listens to the Grateful Dead, she gets depressed.
text_b: None
label:1
prediction:[0.00788694 0.9921131 ]

INFO:tensorflow:prediction_loop marked as finished


I0408 03:31:29.418817 140215859427200 error_handling.py:93] prediction_loop marked as finished


INFO:tensorflow:Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


I0408 03:31:29.419226 140214216939264 error_handling.py:70] Error recorded from infeed: Step was cancelled by an explicit call to `Session::Close()`.


W0408 03:31:29.425776 140215859427200 tf_logging.py:161] Reraising captured error
Exception ignored in: <generator object TPUEstimator.predict at 0x7f862fc329e8>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 2500, in predict
    rendezvous.raise_errors()
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 132, in raise_errors
    six.reraise(typ, value, traceback)
  File "/usr/local/lib/python3.6/dist-packages/six.py", line 693, in reraise
    raise value
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/error_handling.py", line 101, in catch_errors
    yield
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/contrib/tpu/python/tpu/tpu_estimator.py", line 478, in _run_infeed
    session.run(self._enqueue_ops)
  File "/usr/local/lib/python3.6/dist-packages/tensorflow/python/client/session.py", line 929, 

## What's next

* Learn about [Cloud TPUs](https://cloud.google.com/tpu/docs) that Google designed and optimized specifically to speed up and scale up ML workloads for training and inference and to enable ML engineers and researchers to iterate more quickly.
* Explore the range of [Cloud TPU tutorials and Colabs](https://cloud.google.com/tpu/docs/tutorials) to find other examples that can be used when implementing your ML project.

On Google Cloud Platform, in addition to GPUs and TPUs available on pre-configured [deep learning VMs](https://cloud.google.com/deep-learning-vm/),  you will find [AutoML](https://cloud.google.com/automl/)*(beta)* for training custom models without writing code and [Cloud ML Engine](https://cloud.google.com/ml-engine/docs/) which will allows you to run parallel trainings and hyperparameter tuning of your custom models on powerful distributed hardware.
